In [3]:
#imports

import numpy as np
import pandas as pd
import matplotlib as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [5]:
df = pd.read_csv("Project DataSet 4.25.23.csv")

df.head()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,MarketingFlags,Marketing Flags_content,available size options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer
0,1284462.0,Acqua Di Parma,Cologne,Colonia Essenza,1.7 oz/ 50 mL,5.0,13.0,719.0,120.0,120.0,...,TRUE,online only,- 1.7 oz/ 50 mL Eau de Cologne Spray - 3.4 o...,The newest Eau de Cologne from Acqua di Parma ...,no instructions,Alcohol Denat.- Water- Fragrance- Limonene- Hy...,1.0,0.0,0.0,0.0
1,2221588.0,Acqua Di Parma,Body Mist & Hair Mist,Peonia Nobile Hair Mist,1.7oz/50mL,4.0,5.0,800.0,58.0,58.0,...,TRUE,online only,- 1.7oz/50mL Hair Mist - 0.70oz/20mL Eau de...,A new Le Nobili beauty ritual leaves your hair...,Suggested Usage:-Spray over your hair to leave...,Alcohol Denat.- Water- Fragrance- Polyquaterni...,1.0,0.0,0.0,0.0
2,2222917.0,Acqua Di Parma,Body Lotions & Body Oils,Fico di Amalfi Spray Body Lotion,5 oz/150 mL,3.0,2.0,420.0,55.0,55.0,...,TRUE,online only,no options,Fragrance Family: FloralScent Type: Fruity Flo...,Suggested Usage:-Spray generously onto clean s...,-Almond Oil- Olive Oil- and Grapeseed Oil: Hy...,1.0,0.0,0.0,0.0
3,967224.0,Acqua Di Parma,Body Sprays & Deodorant,Colonia Alcohol-Free Deodorant,2.7 oz/ 77 g,3.5,7.0,400.0,50.0,50.0,...,TRUE,online only,no options,What it is:A deodorant stick that wears comfor...,no instructions,Aqua (Water)- Butylene Glycol- Glycerin- Sodiu...,1.0,0.0,0.0,0.0
4,1059500.0,Acqua Di Parma,Cologne,Colonia Intensa,1.7 oz/ 50 mL,4.0,20.0,584.0,120.0,120.0,...,TRUE,online only,- 1.7 oz/ 50 mL Eau de Cologne Spray,Colonia Intensa- the first Eau de Cologne pure...,no instructions,After Shave Balm:-Aloe Vera Gel: Soothes irrit...,1.0,0.0,0.0,0.0


In [6]:
df.shape

(5872, 21)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5872 entries, 0 to 5871
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5866 non-null   float64
 1   brand                    5866 non-null   object 
 2   category                 5866 non-null   object 
 3   name                     5866 non-null   object 
 4   size                     5866 non-null   object 
 5   rating                   5866 non-null   float64
 6   number_of_reviews        5866 non-null   float64
 7   love                     5866 non-null   float64
 8   price                    5866 non-null   float64
 9   value_price              5866 non-null   float64
 10  URL                      5869 non-null   object 
 11  MarketingFlags           5869 non-null   object 
 12  Marketing Flags_content  5869 non-null   object 
 13  available size options   5869 non-null   object 
 14  details                 

## Notice the RangeIndex in the cell above. Each column should have 5871 non-null values due to the 5871 records. Each column currently has 5866, which means 5 records have no data. After looking at the data, those 5 records are all from the previous preprocesing in Excel, with empty cells.


In [8]:
df = df.dropna(axis=0,how='any')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5866 entries, 0 to 5865
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5866 non-null   float64
 1   brand                    5866 non-null   object 
 2   category                 5866 non-null   object 
 3   name                     5866 non-null   object 
 4   size                     5866 non-null   object 
 5   rating                   5866 non-null   float64
 6   number_of_reviews        5866 non-null   float64
 7   love                     5866 non-null   float64
 8   price                    5866 non-null   float64
 9   value_price              5866 non-null   float64
 10  URL                      5866 non-null   object 
 11  MarketingFlags           5866 non-null   object 
 12  Marketing Flags_content  5866 non-null   object 
 13  available size options   5866 non-null   object 
 14  details                 

In [9]:
#checking to see if any columns are missing values
df.columns[df.isna().any()].tolist()

[]

In [10]:
# Filter products based on the specified ingredients
ingredients = ["avobenzone", "ecamsule", "oxybenzone", "octocrylene", "zinc oxide", "titanium dioxide", "octinoxate"]
df_filtered = df[df["ingredients"].str.contains('|'.join(ingredients), case=False)]

# Use TF-IDF to vectorize the ingredients of the products

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_filtered["ingredients"])

#from sklearn.metrics.pairwise import cosine_similarity
#cosine_similarities = cosine_similarity(tfidf_matrix)


# Using KNN as our model
k = 5
neigh = NearestNeighbors(n_neighbors=k, metric='cosine')
neigh.fit(tfidf_matrix)
top_k_indexes = neigh.kneighbors(tfidf_matrix, return_distance=False)


# Define the target products for which recommendations need to be generated
target_products = df_filtered[df_filtered["name"].str.contains("sunscreen|spf", case=False)]
target_products = target_products.sort_values(by=['rating', 'number_of_reviews'], ascending=False)


# Iterate over the target products
for i, product in target_products.iterrows():
    print('\nProduct Name:', product['name'])
    print('Rating:', product['rating'])
    #print('Number of Reviews:', product['number_of_reviews'])
    
    # Get the index of the current product in the cosine similarities matrix
    index = df_filtered.index.get_loc(i)
    
    # Get the top k similar products
    top_k_indexes = neigh.kneighbors(tfidf_matrix[index], n_neighbors=k+1, return_distance=False)
    
    # Remove the index of the current product from the list of similar products
    top_k_indexes = top_k_indexes[0][1:]
    
    # Get the recommended products based on the indexes
    recommended_products = df_filtered.iloc[top_k_indexes]
    
    # Sort the recommended products by rating and number of reviews
    recommended_products = recommended_products.sort_values(by=['rating', 'number_of_reviews'], ascending=False)
    
    # Print the recommended products
    print('Recommended products:')
    for index, recommended_product in recommended_products.iterrows():
        print('- ' + recommended_product['name'], '\n\tRating: ', recommended_product['rating'],'\n\tNumber of Reviews: ', recommended_product['number_of_reviews'])




Product Name: Every Sun Day™ Mineral Sunscreen SPF 50+
Rating: 5.0
Recommended products:
- Glow One Step Further 
	Rating:  4.5 
	Number of Reviews:  4.0
- Premium Beauty Balm SPF 45 
	Rating:  4.0 
	Number of Reviews:  2000.0
- Dis-A-Pore Beauty Balm SPF 50+ 
	Rating:  4.0 
	Number of Reviews:  77.0
- All Nighter Liquid Foundation 
	Rating:  3.5 
	Number of Reviews:  3000.0
- All Nighter Waterproof Full-Coverage Concealer 
	Rating:  3.5 
	Number of Reviews:  355.0

Product Name: Invisible Physical Defense Sunscreen SPF 30
Rating: 5.0
Recommended products:
- City Skin Age Defense Broad Spectrum SPF 50 PA++++ 
	Rating:  4.5 
	Number of Reviews:  236.0
- Broad Spectrum SPF 30 Mineral Sunscreen Lotion for Body 
	Rating:  4.5 
	Number of Reviews:  19.0
- Skinperfect Primer SPF 30 
	Rating:  4.5 
	Number of Reviews:  15.0
- Sun Safety Kit 
	Rating:  4.0 
	Number of Reviews:  25.0
- Superdefense City Block SPF 50+ Daily Energy + Face Protector 
	Rating:  0.0 
	Number of Reviews:  0.0

Produ

Recommended products:
- Glowmotions Glow Body Oil 
	Rating:  4.0 
	Number of Reviews:  340.0
- Açaí Body Power Cream 
	Rating:  4.0 
	Number of Reviews:  193.0
- Bum Bum Sol Oil SPF 30 
	Rating:  4.0 
	Number of Reviews:  109.0
- Açai Body Power Cream Mini 
	Rating:  4.0 
	Number of Reviews:  57.0
- Limited Edition Glowmotions Glow Body Oil 
	Rating:  0.0 
	Number of Reviews:  0.0

Product Name: Forever Young Body Butter with Sea Buckthorn SPF 40 PA+++
Rating: 4.5
Recommended products:
- Sandara Eau de Parfum Travel Spray 
	Rating:  5.0 
	Number of Reviews:  9.0
- No.04 Bois de Balincourt Perfume Oil 
	Rating:  4.5 
	Number of Reviews:  83.0
- No.02 Le Long Fond Perfume Oil 
	Rating:  4.5 
	Number of Reviews:  10.0
- No.09 Vallée de Farney Perfume Oil 
	Rating:  4.0 
	Number of Reviews:  9.0
- No.05 Kandilli Perfume Oil 
	Rating:  3.5 
	Number of Reviews:  8.0

Product Name: The Broad Spectrum SPF 50 UV Protecting Fluid
Rating: 4.5
Recommended products:
- The Hydrating Illuminator 
	Ra

Recommended products:
- 100 percent Pure Argan Oil 
	Rating:  4.5 
	Number of Reviews:  7000.0
- 100 percent Pure Argan Oil Light  
	Rating:  4.5 
	Number of Reviews:  1000.0
- 100 percent Pure Argan Oil Mini 
	Rating:  4.5 
	Number of Reviews:  182.0
- Hanami Eau de Parfum Travel Spray 
	Rating:  4.5 
	Number of Reviews:  15.0
- Argan Daily Moisturizer SPF 47  
	Rating:  4.0 
	Number of Reviews:  1000.0

Product Name: Mattescreen Sunscreen SPF 40
Rating: 4.0
Recommended products:
- James Oil 
	Rating:  5.0 
	Number of Reviews:  4.0
- Original Skin™ Retexturizing Mask with Rose Clay 
	Rating:  4.5 
	Number of Reviews:  1000.0
- Out of Trouble™ 10 Minute Mask to Rescue Problem Skin 
	Rating:  4.5 
	Number of Reviews:  897.0
- Angie Oil 
	Rating:  4.5 
	Number of Reviews:  2.0
- Tilly Oil 
	Rating:  0.0 
	Number of Reviews:  0.0

Product Name: Superscreen Daily Moisturizer Broad Spectrum SPF 40 PA+++
Rating: 4.0
Recommended products:
- The Silk Peony Melting Eye Cream 
	Rating:  4.5 
	Nu

Recommended products:
- Selfie Shield™ Broad Spectrum SPF 38 Dry Oil Primer 
	Rating:  4.5 
	Number of Reviews:  169.0
- Oil Free Facial Lotion 
	Rating:  4.5 
	Number of Reviews:  42.0
- Rènergie Lift Multi-Action Rich Cream with SPF 15 For Dry Skin 
	Rating:  4.5 
	Number of Reviews:  7.0
- Ultra Facial Cream SPF 30 
	Rating:  4.0 
	Number of Reviews:  96.0
- Powerful Wrinkle Reducing Cream Sunscreen Broad Spectrum SPF 30 
	Rating:  4.0 
	Number of Reviews:  9.0

Product Name: Traceless Perfecting Foundation Broad Spectrum SPF 15
Rating: 4.0
Recommended products:
- Studio Skin Flawless 24 Hour Concealer 
	Rating:  4.5 
	Number of Reviews:  696.0
- Skin Long-Wear Weightless Foundation SPF 15 
	Rating:  4.0 
	Number of Reviews:  824.0
- Emotionproof Concealer 
	Rating:  4.0 
	Number of Reviews:  84.0
- Contour Sculpting Powder 
	Rating:  4.0 
	Number of Reviews:  30.0
- Beyond Perfecting Super Concealer Camouflage + 24-Hour Wear - Apricot Color Corrector 
	Rating:  3.5 
	Number of Revi

Recommended products:
- Raven Excellent Body Milk 
	Rating:  5.0 
	Number of Reviews:  3.0
- Vanilla Woods Eau de Parfum Rollerball 
	Rating:  4.5 
	Number of Reviews:  114.0
- Jasmine Neroli Eau de Parfum Rollerball 
	Rating:  4.0 
	Number of Reviews:  34.0
- Jade Facial Roller 
	Rating:  3.5 
	Number of Reviews:  198.0
- Vita Mineral™ Active 7 Eye Gel  
	Rating:  3.5 
	Number of Reviews:  148.0

Product Name: PLAY 100% Mineral Mist SPF 30 with Marigold Extract
Rating: 3.0
Recommended products:
- No.04 Bois de Balincourt Perfume Oil 
	Rating:  4.5 
	Number of Reviews:  83.0
- No.02 Le Long Fond Perfume Oil 
	Rating:  4.5 
	Number of Reviews:  10.0
- The Tea Tree Trio 
	Rating:  4.0 
	Number of Reviews:  34.0
- No.09 Vallée de Farney Perfume Oil 
	Rating:  4.0 
	Number of Reviews:  9.0
- No.05 Kandilli Perfume Oil 
	Rating:  3.5 
	Number of Reviews:  8.0

Product Name: CC+ Cream Oil-Free Matte with SPF 40 Mini
Rating: 3.0
Recommended products:
- CC+ Cream Oil-Free Matte with SPF 40 
	R